In [3]:


from commonfunctions import *
inpath = '../inputs/'
outpath = '../outputs/segmentation/'
import time


In [4]:
def get_lengthes(img, offset=1):
    cols = img.shape[1]
    rows = img.shape[0]
    cur = 0
    hist = np.zeros((rows,rows), dtype=np.uint32)
    for i in range(0,cols):
        flag = False
        begin = 0
        blackRuns = []
        whiteRuns = []
        runtype = 0
        while begin < rows and img[begin,i] == 1:
            begin += 1
        for j in range(begin, rows):
            if img[j,i] == runtype:
                cur += 1
            else:
                if runtype == 0:
                    blackRuns.append(cur)
                else:
                    whiteRuns.append(cur)
                cur = 1
                runtype = img[j, i]
                if flag:
                    hist[blackRuns[-1], whiteRuns[-1]] += 1
                flag = True
        cur = 0
    mx = np.max(hist)
    ind = np.where(hist == mx)
    thickness = ind[0][0]
    distance = ind[1][0]
    return thickness, distance

In [5]:
 def get_candidates_lines(bimg, thickness):
    fimg = np.copy(bimg)
    candidates = [] # Contains list of candidate staffs lines (row, begin, height)
    cols = fimg.shape[1]
    rows = fimg.shape[0]
    cur = 0
    t2 = thickness+2
    t1 = abs(thickness-2)
    flag = False
    for i in range(cols):
        for j in range(rows):
            if fimg[j,i] == 0 and flag == False:
                beg = j
                flag = True
            elif fimg[j,i] == 1 and flag == True:
                flag = False
                if j-beg > t2 or j-beg < t1:
                    fimg[beg:j, i] = 1
                else:
                    candidates.append((i, beg, j-beg))
    return fimg, candidates


In [6]:
def remove_thin_lines(v, csl, thickness):
    cols = csl.shape[1]
    rows = csl.shape[0]
    candidates = v.copy()
    img = np.copy(csl)
    eliminated = []
    for i in candidates:
        col, begin, length = i
        maxWidthLeft = 0
        maxWidthRight = 0
        wLeft = 0
        wRight = 0
        while col+wLeft+1 < cols and begin < i[1]+length:
            if csl[begin, col+wLeft+1] == 0:
                wLeft += 1
            else:
                begin += 1
                maxWidthLeft = max(maxWidthLeft, wLeft)
                wLeft = 0
        maxWidthLeft = max(maxWidthLeft, wLeft)
        begin = i[1]
        while col-wRight-1 >= 0 and begin < i[1]+length:
            if csl[begin, col-wRight-1] == 0:
                wRight += 1
            else:
                begin += 1
                maxWidthRight = max(maxWidthRight, wRight)
                wRight = 0
        maxWidthRight = max(maxWidthRight, wRight)
        width = maxWidthRight + maxWidthLeft +1
        if(width < 2*thickness):
            img[i[1]:i[1]+i[2], i[0]] = 1
            v.remove(i)
            eliminated.append(i)
    return img, v, eliminated

In [7]:
def remove_lonely_lines(v, filteredImg, staffHeight, spaceHeight, eliminated):
    verConnected = False
    horConnected = False
    c = v.copy()
    img = np.copy(filteredImg)
    cols = filteredImg.shape[1]
    rows = filteredImg.shape[0]
    staffs = np.ones(img.shape)

    for i in c:
        verConnected = False
        horConnected = False
        
        col, begin, length = i
        
        if col-1>=0 and np.any(filteredImg[begin:begin+length , col-1] == 0):
            horConnected = True
        if col+1<cols and np.any(filteredImg[begin:begin+length ,col+1] == 0):
            horConnected = True
            
        start = max(begin-(spaceHeight+staffHeight), 0)
        end = min(rows, begin+length+spaceHeight+staffHeight-1)
        if np.any(filteredImg[start:begin, col] == 0) or np.any(filteredImg[begin+length:end, col] == 0):
            verConnected = True
            
        if not (verConnected and horConnected):
            img[i[1]:i[1]+i[2], i[0]] = 1
            v.remove(i)
            eliminated.append(i)
    for i in v:
        staffs[i[1]:i[1]+i[2],i[0]] = 0
    return staffs, v, eliminated

In [8]:
def add_false_negative_lines(v, filteredImg, staffHeight, spaceHeight, eliminated):
    verConnected = False
    horConnected = False
    c = v.copy()
    eliminated.copy()
    img = np.copy(filteredImg)
    cols = filteredImg.shape[1]
    rows = filteredImg.shape[0]
    
    for i in eliminated:
        verConnected = False
        horConnected = False
        
        col, begin, length = i
        
        if col-1>=0 and np.any(filteredImg[begin:begin+length , col-1]):
            horConnected = True
        if col+1<cols and np.any(filteredImg[begin:begin+length ,col+1]):
            horConnected = True
        start = max(begin-(spaceHeight+staffHeight), 0)
        end = min(rows, begin+length+spaceHeight+staffHeight)
        if np.any(filteredImg[start:begin, col]) or np.any(filteredImg[begin+length:end, col]):
            verConnected = True
            
        if verConnected and horConnected:
            img[i[1]:i[1]+i[2], i[0]] = 0
            v.append(i)
            
    return img, v

In [9]:
def binraization(img,n=8,t=15):

    outputimg = np.zeros(img.shape)
    intimg = np.zeros(img.shape)
    h = img.shape[1]
    w = img.shape[0]
    s= min(w,h)//n
    count = s**2
    img = np.pad(img,s,"constant")
    intimg = np.cumsum(img ,axis =1)
    intimg = np.cumsum(intimg ,axis =0)
    a = np.roll(intimg,-s//2,axis =0)
    a = np.roll(a,-s//2,axis =1)
    a[:,-s//2:]=a[-s//2-1,-s//2-1]
    a[-s//2:,:]=a[-s//2-1,-s//2-1]
    b = np.roll(intimg,s//2+1,axis =0)
    b = np.roll(b,-s//2,axis =1)
    b[0:s//2+1,:]=0
    b[:,-s//2:]=0
    
    c = np.roll(intimg,s//2+1,axis =1)
    c = np.roll(c,-s//2,axis =0)
    c[:,0:s//2+1]=0
    c[-s//2:,:]=0
    
    d = np.roll(intimg,s//2+1,axis =0)
    d = np.roll(d,s//2+1,axis =1)
    d[0:s//2+1,:]=0
    d[:,0:s//2+1]=0

    sum = (a-b-c+d)*(100-t)/100
    outputimg = np.where(img>sum/count, 1, 0)
    return outputimg[s:-s,s:-s]

In [10]:
def segement_hand_written(img):
    h , w = img.shape 
    window , width = get_dimensions(w,h)
    fimg = median(img)
    glares = (fimg > 195)
    bimg = binraization(fimg,window,7)
    output = (bimg ) - (glares)
    output = median(output)
    output = binary_dilation(1-output)
    vse = np.array([[0,1,0],
                    [0,1,0],
                    [0,1,0]      ])

    hse = np.array([[0,0,0],
                    [1,1,1],
                    [0,0,0]
    ])
    output = binary_dilation(output , vse)
    output = binary_dilation(output , hse)
    output = 1 - output
    return output #the reuslt is binary image

In [11]:
def remove_staff_lines(bimg , t , d,hw):
    csl , c = get_candidates_lines(bimg,t) 
    fcsl , c , e = remove_thin_lines(c,csl,t)
    fcsl , c , e = remove_lonely_lines(c,csl,t,d,e)
    fcsl , c  = add_false_negative_lines(c,csl,t,d,e)
    staffless = (bimg-fcsl)
    if(hw):
        staffless = median(staffless)
        staffless = binary_closing(staffless,np.ones((d//2,d//2)))
        staffless = median(staffless)
    result = ((staffless)).astype("uint8")
    return result #the result is a uint8 bw image

In [12]:
tcimgs , tcfiles = load_images_from_folder(inpath)
hwimgs , hwfiles = load_images_from_folder(inpath+"testcases/handwritten")
cimgs , cfiles = load_images_from_folder(inpath+"testcases/camera")
simgs , sfiles = load_images_from_folder(inpath+"testcases/scanned")

../inputs/
../inputs/testcases/handwritten
../inputs/testcases/camera
../inputs/testcases/scanned


In [16]:
path = outpath
files = tcfiles
images = tcimgs
start_time = time.time()
for i , img in enumerate(images) :
    h , w = img.shape 
    window , width = get_dimensions(w,h)
    hist = histogram(img,nbins=256,source_range='dtype')
    hist_acc =np.cumsum(hist[0])
    tratio = ((hist_acc[255]-hist_acc[206] + hist_acc[49]-hist_acc[0]) / hist_acc[-1])*100
  
    if ( tratio < 4):
        bimg = segement_hand_written(img)
    elif ( 75 > tratio > 4):
        bimg = binraization(img,window,35)
    else: 
        bimg = (img > 150).astype("uint8")
    io.imsave(path+"/binary"+files[i],(bimg*255).astype("uint8"))
    """ if(width): 
        img = resize(img,width=w//2)
        bimg = resize(bimg,width=w//2) """
    t,d = get_lengthes(bimg)
    img_isolated = remove_staff_lines(bimg,t,d , tratio < 4 )
    io.imsave(path+"/isolated"+files[i],img_isolated)

    contours, _ = cv2.findContours(img_isolated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    temp_img = img.copy()
    backtorgb = cv2.cvtColor(temp_img,cv2.COLOR_GRAY2RGB)
    for contour in contours:
        x,y,w,h = cv2.boundingRect(contour)
        if(w > d//2 and h > d):
            cv2.rectangle(backtorgb,(x,y),(x+w,y+h),(50,100,200),3)
    io.imsave(path+"/contours"+files[i],backtorgb)
print("--- %s seconds ---" % (time.time() - start_time))
    

--- 257.5789759159088 seconds ---


In [ ]:
img = hwimgs[4]
bimg = segement_hand_written(img)
io.imsave(inpath+"testcases/handwritten/binary"+hwfiles[i],bimg)
img_isolated = remove_staff_lines(bimg)
io.imsave(inpath+"testcases/handwritten/isolated"+hwfiles[i],img_isolated)
contours, _ = cv2.findContours(result, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
temp_img = img.copy()
for contour in contours:
    x,y,w,h = cv2.boundingRect(contour)
    if(w > d//2 and h > d):
        cv2.rectangle(temp_img,(x,y),(x+w,y+h),(255,0,0),3)
io.imsave(inpath+"testcases/handwritten/isolated"+hwfiles[i],temp_img)


#--- 204.8634433746338 seconds ---
#--- 435.44309854507446 seconds ---